In [50]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv('ner_d.csv', encoding = "ISO-8859-1")
print(len(df))
# df = df[:100000]
#50000 and above, 80000,100000
df.head()

1048575


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [52]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(47959, 35177, 17)

In [53]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,402
1,B-eve,308
2,B-geo,37644
3,B-gpe,15870
4,B-nat,201
5,B-org,20143
6,B-per,16990
7,B-tim,20333
8,I-art,297
9,I-eve,253


In [54]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [55]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func_x = lambda s: [w for w in s['Word'].values.tolist()]
        agg_func_y = lambda s: [[p, t] for p, t in zip(s['POS'].values.tolist(),s['Tag'].values.tolist())]
        
        self.grouped_x = self.data.groupby('Sentence #').apply(agg_func_x)
        self.grouped_y = self.data.groupby('Sentence #').apply(agg_func_y)
        self.x=[s for s in self.grouped_x]
        self.y=[s for s in self.grouped_y]
        
#     def get_next(self):
#         try: 
#             s = self.grouped['Sentence: {}'.format(self.n_sent)]
#             self.n_sent += 1
#             return s 
#         except:
#             return None
getter = SentenceGetter(df)
sentence_x = getter.x
sentence_y=getter.y

In [56]:
print(sentence_x[1])
print(sentence_y[1])
assert len(sentence_x[1])==len(sentence_y[1])

['Iranian', 'officials', 'say', 'they', 'expect', 'to', 'get', 'access', 'to', 'sealed', 'sensitive', 'parts', 'of', 'the', 'plant', 'Wednesday', ',', 'after', 'an', 'IAEA', 'surveillance', 'system', 'begins', 'functioning', '.']
[['JJ', 'B-gpe'], ['NNS', 'O'], ['VBP', 'O'], ['PRP', 'O'], ['VBP', 'O'], ['TO', 'O'], ['VB', 'O'], ['NN', 'O'], ['TO', 'O'], ['JJ', 'O'], ['JJ', 'O'], ['NNS', 'O'], ['IN', 'O'], ['DT', 'O'], ['NN', 'O'], ['NNP', 'B-tim'], [',', 'O'], ['IN', 'O'], ['DT', 'O'], ['NNP', 'B-org'], ['NN', 'O'], ['NN', 'O'], ['VBZ', 'O'], ['VBG', 'O'], ['.', 'O']]


In [57]:

################################

def word2features(sent, i):
    word = sent[i]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True
    
    
    #extract features from 2 last words
    if i > 1:
        word1 = sent[i-2][0]
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper()
        }) 
        
        
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True
    
    
    #extract features from 2 next words
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper()
        })
    
    
    
    
    features['position']=np.sin(i)
    return features


def sent2features(sent_x):
    return [word2features(sent_x, i) for i in range(len(sent_x))]
def sent2labels(sent_y):
    return [label for postag, label in sent_y]
def sent2postag(sent_y):
    return [postag for postag, label in sent_y]


In [58]:
X = [sent2features(s) for s in sentence_x]
y1 = [sent2labels(s) for s in sentence_y]
y2 = [sent2postag(s) for s in sentence_y]


In [59]:
assert len(y2[0])==len(y1[0])==len(X[0])

In [60]:
train_length=int(len(X)*0.75)

In [61]:
x_train_set=X[0:train_length]
y1_train_set=y1[0:train_length]
y2_train_set=y2[0:train_length]

x_test_set=X[train_length:]
y1_test_set=y1[train_length:]
y2_test_set=y2[train_length:]

In [62]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=300,
    all_possible_transitions=True
)
try:
    crf.fit(x_train_set, y1_train_set)
except:
    pass

In [63]:
y_pred = crf.predict(x_test_set)

#calculate accuracy
t_pred=[]
t_test=[]
for x in y_pred:
    t_pred.extend(x)
    
for x in y1_test_set:
    t_test.extend(x)

n_test=[]
n_pred=[]
for i in range(len(t_pred)):
    if t_pred[i]==t_test[i] and t_pred[i]=='O':
        pass
    else:
        n_test.append(t_test[i])
        n_pred.append(t_pred[i])

#print(f1_score(t_pred, t_test, average="micro"))
n_pred=np.array(n_pred)
n_test=np.array(n_test)
right=len(n_test[n_pred==n_test])
alles=len(n_test)
print(right)
print(alles)
print(right/alles)



33033
41420
0.7975132786093675


# Bewerbung

In [64]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-art  -> I-art   7.828138
I-eve  -> I-eve   6.996550
B-nat  -> I-nat   6.948121
B-geo  -> I-geo   6.916702
I-art  -> I-art   6.484916
B-eve  -> I-eve   6.374049
B-per  -> I-per   6.130867
I-tim  -> I-tim   5.979728
I-gpe  -> I-gpe   5.826919
B-tim  -> I-tim   5.746286
I-geo  -> I-geo   5.630867
B-org  -> I-org   5.592338
I-org  -> I-org   5.510719
B-gpe  -> I-gpe   5.144799
I-per  -> I-per   4.951164
I-nat  -> I-nat   3.601268
O      -> O       2.609024
B-geo  -> B-tim   2.048582
O      -> B-eve   1.845022
O      -> B-per   1.843359

Top unlikely transitions:
I-per  -> B-geo   -3.448911
B-tim  -> I-org   -3.524673
I-org  -> I-geo   -3.599099
B-geo  -> I-org   -3.607279
I-org  -> I-per   -3.723078
I-org  -> I-tim   -3.814219
B-org  -> B-org   -3.920244
B-tim  -> B-tim   -4.090297
O      -> I-eve   -4.132584
B-gpe  -> I-geo   -4.264852
I-org  -> B-org   -4.396174
O      -> I-art   -4.555218
B-gpe  -> I-org   -4.695370
O      -> I-per   -5.149958
B-gpe  -> B-gpe  

In [65]:
import joblib

# save
joblib.dump(crf, "model_crf.pkl") 

# load
crf2 = joblib.load("model_crf.pkl")

# question test

In [66]:
question="Who is the screenwriter of The Masked Gang: Cyprus?"
question=question.split(' ')
print(question)
q_x=[sent2features(question)]
q_y = crf2.predict(q_x)
q_y

['Who', 'is', 'the', 'screenwriter', 'of', 'The', 'Masked', 'Gang:', 'Cyprus?']


[['O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'I-org']]

# try more data

In [67]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [68]:
pd.set_option('display.max_columns', None)

In [69]:
n_df = pd.read_csv('ner.csv',encoding='cp1252')
# n_df = pd.read_csv('ner.csv', encoding = "ISO-8859-1")
print(len(n_df))
# df = df[:100000]
#50000 and above, 80000,100000
n_df.head()

#DELETE LINE 281837 BUGS HERE!!!

1050795


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [70]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func_x = lambda s: [w for w in s['word'].values.tolist()]
        agg_func_y = lambda s: [[p, t] for p, t in zip(s['pos'].values.tolist(),s['tag'].values.tolist())]
        
        self.grouped_x = self.data.groupby('sentence_idx').apply(agg_func_x)
        self.grouped_y = self.data.groupby('sentence_idx').apply(agg_func_y)
        self.x=[s for s in self.grouped_x]
        self.y=[s for s in self.grouped_y]
        
#     def get_next(self):
#         try: 
#             s = self.grouped['Sentence: {}'.format(self.n_sent)]
#             self.n_sent += 1
#             return s 
#         except:
#             return None
getter = SentenceGetter(n_df)
big_sentence_x = getter.x
big_sentence_y=getter.y

In [71]:
def word2features(sent, i):
    word = str(sent[i])
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    

    if i > 0:
        word1 = str(sent[i-1])[0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True

        
    if i > 1:
        word1 = str(sent[i-2])[0]
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper()
        }) 
        
        
    if i < len(sent)-1:
        word1 = str(sent[i+1])[0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True
        
        
    
    #extract features from 2 next words
    if i < len(sent)-2:
        word1 = str(sent[i+2])[0]
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper()
        })
    


    features['position']=np.sin(i)
    return features

def sent2features(sent_x):
    return [word2features(sent_x, i) for i in range(len(sent_x))]
def sent2labels(sent_y):
    return [label for postag, label in sent_y]
def sent2postag(sent_y):
    return [postag for postag, label in sent_y]


In [72]:
X = [sent2features(s) for s in big_sentence_x]
y1 = [sent2labels(s) for s in big_sentence_y]
y2 = [sent2postag(s) for s in big_sentence_y]


In [73]:
train_length=int(len(X)*0.75)

In [74]:
x_train_set=X[0:train_length]
y1_train_set=y1[0:train_length]
y2_train_set=y2[0:train_length]

x_test_set=X[train_length:]
y1_test_set=y1[train_length:]
y2_test_set=y2[train_length:]

In [75]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=300,
    all_possible_transitions=True
)
try:
    crf.fit(x_train_set, y1_train_set)
except:
    pass

In [76]:
y_pred = crf.predict(x_test_set)

#calculate accuracy
t_pred=[]
t_test=[]
for x in y_pred:
    t_pred.extend(x)
    
for x in y1_test_set:
    t_test.extend(x)

n_test=[]
n_pred=[]
for i in range(len(t_pred)):
    if t_pred[i]==t_test[i] and t_pred[i]=='O':
        pass
    else:
        n_test.append(t_test[i])
        n_pred.append(t_pred[i])

#print(f1_score(t_pred, t_test, average="micro"))
n_pred=np.array(n_pred)
n_test=np.array(n_test)
right=len(n_test[n_pred==n_test])
alles=len(n_test)
print(right)
print(alles)
print(right/alles)



24094
30525
0.7893202293202293


In [77]:
import joblib

# save
joblib.dump(crf, "model_crf.pkl") 

# load
crf = joblib.load("model_crf.pkl")

In [78]:
question="Who is the screenwriter of The Masked Gang: Cyprus?"
question=question.split(' ')
print(question)
q_x=[sent2features(question)]
q_y = crf2.predict(q_x)
q_y

['Who', 'is', 'the', 'screenwriter', 'of', 'The', 'Masked', 'Gang:', 'Cyprus?']


[['O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'I-org']]

# with pos

In [79]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [80]:
pd.set_option('display.max_columns', None)

In [81]:
n_df = pd.read_csv('ner.csv',encoding='cp1252')
# n_df = pd.read_csv('ner.csv', encoding = "ISO-8859-1")
print(len(n_df))
# df = df[:100000]
#50000 and above, 80000,100000
n_df.head()

#DELETE LINE 281837 BUGS HERE!!!

1050795


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [82]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func_x = lambda s: [[w,p] for w,p in zip(s['word'].values.tolist(),s['pos'].values.tolist())]
        agg_func_y = lambda s: [t for t in s['tag'].values.tolist()]
        
        self.grouped_x = self.data.groupby('sentence_idx').apply(agg_func_x)
        self.grouped_y = self.data.groupby('sentence_idx').apply(agg_func_y)
        self.x=[s for s in self.grouped_x]
        self.y=[s for s in self.grouped_y]
        
#     def get_next(self):
#         try: 
#             s = self.grouped['Sentence: {}'.format(self.n_sent)]
#             self.n_sent += 1
#             return s 
#         except:
#             return None
getter = SentenceGetter(n_df)
big_sentence_x = getter.x
big_sentence_y=getter.y

In [83]:
def word2features(sent, i):
    word = str(sent[i][0])
    tag=str(sent[i][1])
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.tag':tag
    }
    

    if i > 0:
        word1 = str(sent[i-1][0])
        tag1=str(sent[i-1][1])
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.tag':tag1
        })
    else:
        features['BOS'] = True

        
    if i > 1:
        word1 = str(sent[i-2][0])
        tag1 = str(sent[i-2][1])
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:word.tag':tag1
        }) 
        
        
    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
        tag1 = str(sent[i+1][1])
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.tag':tag1
        })
    else:
        features['EOS'] = True
        
        
    
    #extract features from 2 next words
    if i < len(sent)-2:
        word1 = str(sent[i+2][0])
        tag1 = str(sent[i+2][1])
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:word.tag': tag1
        })
    


    features['position']=np.sin(i)
    return features

def sent2features(sent_x):
    return [word2features(sent_x, i) for i in range(len(sent_x))]
def sent2labels(sent_y):
    return [label for label in sent_y]


In [84]:
X = [sent2features(s) for s in big_sentence_x]
y = [sent2labels(s) for s in big_sentence_y]


In [85]:
train_length=int(len(X)*0.75)

In [86]:
x_train_set=X[0:train_length]
y_train_set=y[0:train_length]

x_test_set=X[train_length:]
y_test_set=y[train_length:]


In [87]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=300,
    all_possible_transitions=True
)
try:
    crf.fit(x_train_set, y_train_set)
except:
    pass

In [88]:
y_pred = crf.predict(x_test_set)

#calculate accuracy
t_pred=[]
t_test=[]
for x in y_pred:
    t_pred.extend(x)
    
for x in y_test_set:
    t_test.extend(x)

n_test=[]
n_pred=[]
for i in range(len(t_pred)):
    if t_pred[i]==t_test[i] and t_pred[i]=='O':
        pass
    else:
        n_test.append(t_test[i])
        n_pred.append(t_pred[i])

#print(f1_score(t_pred, t_test, average="micro"))
n_pred=np.array(n_pred)
n_test=np.array(n_test)
right=len(n_test[n_pred==n_test])
alles=len(n_test)
print(right)
print(alles)
print(right/alles)



24619
30366
0.8107422775472568


In [89]:
import joblib

# save
joblib.dump(crf, "model_crf.pkl") 

# load
crf2 = joblib.load("model_crf.pkl")

In [90]:
question="Who is the screenwriter of The Masked Gang: Cyprus?"
question=question.split(' ')
print(question)
q_x=[sent2features(question)]
q_y = crf2.predict(q_x)
q_y

['Who', 'is', 'the', 'screenwriter', 'of', 'The', 'Masked', 'Gang:', 'Cyprus?']


[['O', 'O', 'O', 'O', 'O', 'B-art', 'I-art', 'I-art', 'I-art']]

# check repeating sentences

In [43]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter


In [44]:
df = pd.read_csv('ner_d.csv', encoding = "ISO-8859-1")
df = df.fillna(method='ffill')
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func_x = lambda s: [w for w in s['Word'].values.tolist()]
        agg_func_y = lambda s: [[p, t] for p, t in zip(s['POS'].values.tolist(),s['Tag'].values.tolist())]
        
        self.grouped_x = self.data.groupby('Sentence #').apply(agg_func_x)
        self.grouped_y = self.data.groupby('Sentence #').apply(agg_func_y)
        self.x=[s for s in self.grouped_x]
        self.y=[s for s in self.grouped_y]

getter = SentenceGetter(df)
sentence_x = getter.x
sentence_y=getter.y

In [45]:
n_df = pd.read_csv('ner.csv',encoding='cp1252')
n_df.head()
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func_x = lambda s: [w for w in s['word'].values.tolist()]
        agg_func_y = lambda s: [[p, t] for p, t in zip(s['pos'].values.tolist(),s['tag'].values.tolist())]
        
        self.grouped_x = self.data.groupby('sentence_idx').apply(agg_func_x)
        self.grouped_y = self.data.groupby('sentence_idx').apply(agg_func_y)
        self.x=[s for s in self.grouped_x]
        self.y=[s for s in self.grouped_y]
        
getter = SentenceGetter(n_df)
big_sentence_x = getter.x
big_sentence_y=getter.y
#DELETE LINE 281837 BUGS HERE!!!

In [46]:
# repeat=0
# for i in range(len(sentence_x)):
#     for j in range(len(big_sentence_x)):
#         if sentence_x[i]==big_sentence_x[j]:
#             print(sentence_x[i])
#             repeat+=1
# print(repeat)


In [47]:
print(len(sentence_x))
print(len(big_sentence_x))

47959
35177


# fix this bug

In [48]:
def delete_repeated_sentence(sentence,sentence_list,y_sentence_list):
    for i in range(len(sentence_list)):
        if sentence_list[i]==sentence:
            del sentence_list[i],y_sentence_list[i]
            break
    return

In [49]:
for i in range(len(sentence_x)):
    delete_repeated_sentence(sentence_x[i],big_sentence_x,big_sentence_y)
    
for i in range(len(big_sentence_x)):
    delete_repeated_sentence(big_sentence_x[i],sentence_x,sentence_y)

    
print(len(sentence_x))
print(len(big_sentence_x))

47959
12924
